In [3]:
import os
import json
import pandas as pd
import traceback

In [4]:
from langchain_openai import ChatOpenAI

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
KEY=os.getenv("OPENAI_API_KEY")

In [7]:
llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)

In [8]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x1546a78c0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x155000440>, root_client=<openai.OpenAI object at 0x1546a5010>, root_async_client=<openai.AsyncOpenAI object at 0x1550001a0>, temperature=0.5, model_kwargs={}, openai_api_key=SecretStr(''), stream_usage=True)

In [13]:
# ✅ Modern LangChain v1.x compatible version

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import PyPDF2

# Step 1: Initialize the OpenAI Chat Model
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

# Step 2: Create a prompt template
prompt = ChatPromptTemplate.from_template(
    "Write a short poem about {topic}"
)

# Step 3: Combine them into a chain
chain = prompt | model | StrOutputParser()

# Step 4: Run it
result = chain.invoke({"topic": "coding"})
print(result)



AuthenticationError: Error code: 401 - {'error': {'message': "You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [18]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [14]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [15]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [21]:
quiz_chain = quiz_generation_prompt | llm | StrOutputParser()

In [22]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [23]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [25]:
review_chain= quiz_evaluation_prompt | llm | StrOutputParser()

In [27]:
from langchain_core.runnables import RunnablePassthrough

# Step 1: Define each chain
# quiz_chain and review_chain must be Runnable objects (e.g. prompt | llm | parser)

# Step 2: Combine them properly
generate_evaluate_chain = (
    {
        "quiz": quiz_chain,
        # Pass through the same input to next chain
        "review": lambda x: review_chain.invoke({
            "quiz": x["quiz"],
            "text": x["text"],
            "subject": x["subject"],
            "tone": x["tone"],
            "response_json": x["response_json"],
        })
    }
)

In [36]:
file_path="/Users/aryanajmani/mcqgen/data.txt"

In [37]:
file_path

'/Users/aryanajmani/mcqgen/data.txt'

In [38]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [40]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[5][6] The synonym self-teaching computers was also used in this time period.[7][8]

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[9] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[10] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[9] Other researchers who have studied human cognitive systems contribute

In [41]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [47]:
NUMBER=5 
SUBJECT="machine learning"
TONE="simple"

In [46]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

response = model.invoke("Write a short poem about coding")

print(response)

AuthenticationError: Error code: 401 - {'error': {'message': "You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [48]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

NameError: name 'cb' is not defined

In [49]:
response

NameError: name 'response' is not defined

In [50]:
response.get("quiz")

NameError: name 'response' is not defined

In [51]:
json.loads(quiz)

NameError: name 'quiz' is not defined

In [52]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

NameError: name 'quiz' is not defined

In [53]:
quiz_table_data

[]

In [54]:
quiz=pd.DataFrame(quiz_table_data)

ModuleNotFoundError: No module named 'numpy.rec'

In [55]:
quiz.to_csv("machine_learning.csv",index=False)

NameError: name 'quiz' is not defined